In [1]:
import sys
# add path for my packages
sys.path.append('C:\\Users\\katsuya2\\OneDrive - University of Illinois - Urbana\\Documents\\Python Scripts\\my_packages')
import re
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import ast
from scipy import optimize
from scipy.optimize import curve_fit
from scipy.stats import norm
import scipy.stats as stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from tqdm import tqdm
from pathlib import Path
from glob import glob
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, make_scorer, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor


%precision 3
%matplotlib inline

# use my matplotlib style
style_p = Path("C:/Users/katsuya2/OneDrive - University of Illinois - Urbana/Documents/Python Scripts/my_packages")
plt.style.use(Path(style_p, 'my_rc_setting.mplstyle'))

In [10]:
p = Path("C:/Users/katsuya2/OneDrive - University of Illinois - Urbana/Documents/Python Scripts/input_data/SPP2023")

# all glass data
# glass_data = pd.read_csv(Path(p, '090723_glass_training.csv'))
# glass_data['ratio'] = glass_data['length'] / glass_data['width']
# glass_data = glass_data.drop(['corrcoef'], axis=1)
# glass_data['volume'] = np.pi*(glass_data['width']**2)*glass_data['length']/4 - np.pi*(glass_data['width']**3)/12


glass_data = pd.read_csv(Path(p, '090723_glass_training.csv'))
glass_data = glass_data.drop(['corrcoef'], axis=1)
original_data = pd.read_csv(Path(p, 'New_SPP.csv'))
combined_data = pd.concat([glass_data, original_data], axis=0)
combined_data['volume'] = np.pi*(combined_data['width']**2)*combined_data['length']/4 - np.pi*(combined_data['width']**3)/12
combined_data['ratio'] = combined_data['length'] / combined_data['width']


combined_data = combined_data[(combined_data['width'] < 60) & (combined_data['length'] < 130)]
combined_data = combined_data.reset_index(drop=True)

# all ITO data
ito_data = pd.read_csv(Path(p, '111623_ito_training.csv'))
ito_data['ratio'] = ito_data['length'] / ito_data['width']
ito_data = ito_data.drop(['corrcoef'], axis=1)

# # further control the dataset so that all aspect ratio in glass can fall into the range of ITO's ratio
# ito_ratio_min = ito_data['ratio'].min()
# ito_ratio_max = ito_data['ratio'].max()

# glass_data = glass_data[(glass_data['ratio'] > ito_ratio_min) & (glass_data['ratio'] < ito_ratio_max)]
# glass_data = glass_data.reset_index(drop=True)
# print(glass_data.shape)

# # further control the dataset so that all width and length in glass can fal into the range of ITO's
# ito_width_min = ito_data['width'].min()
# ito_width_max = ito_data['width'].max()
# ito_length_min = ito_data['length'].min()
# ito_length_max = ito_data['length'].max()

# glass_data = glass_data[(glass_data['width'] > ito_width_min) & (glass_data['width'] < ito_width_max)]
# glass_data = glass_data[(glass_data['length'] > ito_length_min) & (glass_data['length'] < ito_length_max)]
# glass_data = glass_data.reset_index(drop=True)
# print(glass_data.shape)

# Randomly sample 1000 rows from each dataframe
# glass_data = glass_data.sample(n=1000, random_state=42)
# glass_data = glass_data.reset_index(drop=True)
# ito_data = ito_data.sample(n=1000, random_state=42)
# ito_data = ito_data.reset_index(drop=True)
# print(glass_data.shape)

combined_data.shape

(1701, 7)

In [4]:
# define testing data from actual experiments
# exp_glass_data = pd.read_csv(Path(p, '100323_ZJ_exp_glass_final.csv'))
exp_glass_data = pd.read_csv(Path(p, 'SPP_AuNRs_exp_short.csv'))
exp_ito_data = pd.read_csv(Path(p, '100323_ZJ_exp_ito_final.csv'))
exp_ito_data = exp_ito_data.drop(47)
exp_ito_data = exp_ito_data.reset_index(drop=True)

# control distribution
# exp_glass_data = exp_glass_data[(exp_glass_data['width'] < 28) & (exp_glass_data['length'] < 100) & (exp_glass_data['length'] > 70)] 
# exp_glass_data = exp_glass_data[(exp_glass_data['length'] > 80) & (exp_glass_data['length'] < 95)] 
# exp_glass_data = exp_glass_data[(exp_glass_data['width'] > 28) & (exp_glass_data['width'] < 31)] 
# exp_glass_data = exp_glass_data.reset_index(drop=True)

exp_ito_data = exp_ito_data[(exp_ito_data['length'] > 80) & (exp_ito_data['length'] < 95)] 
exp_ito_data = exp_ito_data[(exp_ito_data['width'] > 28) & (exp_ito_data['width'] < 31)] 
exp_ito_data = exp_ito_data.reset_index(drop=True)

# create the specific number of data
# exp_glass_data = exp_glass_data.sample(50, random_state=42)
exp_ito_data = exp_ito_data.sample(30, random_state=42)

# X_exp_glass = exp_glass_data.drop(['exp_ene', 'exp_dfs', 'imax', 'width', 'length', 'fitting', 'aspect_ratio', 'volume', 'surface_area'], axis=1)
X_exp_glass = exp_glass_data.drop(['imax', 'width', 'length'], axis=1)
yw_exp_glass = exp_glass_data['width']
yl_exp_glass = exp_glass_data['length']

X_exp_ito = exp_ito_data.drop(['exp_ene', 'exp_dfs', 'imax', 'width', 'length', 'fitting', 'ratio', 'volume', 'surface_area'], axis=1)
yw_exp_ito = exp_ito_data['width']
yl_exp_ito = exp_ito_data['length']

print(X_exp_glass.shape, X_exp_ito.shape)

(26, 2) (30, 2)


In [ ]:
p = Path("C:/Users/katsuya2/OneDrive - University of Illinois - Urbana/Documents/Python Scripts/input_data/SPP2023")

# all glass data
# glass_data = pd.read_csv(Path(p, '090723_glass_training.csv'))
# glass_data['ratio'] = glass_data['length'] / glass_data['width']
# glass_data = glass_data.drop(['corrcoef'], axis=1)
# glass_data['volume'] = np.pi*(glass_data['width']**2)*glass_data['length']/4 - np.pi*(glass_data['width']**3)/12

glass_data = pd.read_csv(Path(p, '090723_glass_training.csv'))
glass_data = glass_data.drop(['corrcoef'], axis=1)

original_data = pd.read_csv(Path(p, 'New_SPP.csv'))
glass_data = pd.concat([glass_data, original_data], axis=0)
glass_data['volume'] = np.pi*(glass_data['width']**2)*glass_data['length']/4 - np.pi*(glass_data['width']**3)/12
glass_data['ratio'] = glass_data['length'] / glass_data['width']
glass_data = glass_data[(glass_data['width'] < 38) & (glass_data['length'] < 130)]
glass_data = glass_data.reset_index(drop=True)
print(glass_data.shape)

In [ ]:
p = Path("C:/Users/katsuya2/OneDrive - University of Illinois - Urbana/Documents/Python Scripts/input_data/SPP2023")

# all ITO data
ito_data = pd.read_csv(Path(p, '111623_ito_training.csv'))
ito_data['ratio'] = ito_data['length'] / ito_data['width']
ito_data = ito_data.drop(['corrcoef'], axis=1)
print(ito_data.shape)

# further control the dataset so that all aspect ratio in glass can fall into the range of ITO's ratio
ito_ratio_min = ito_data['ratio'].min()
ito_ratio_max = ito_data['ratio'].max()

glass_data = glass_data[(glass_data['ratio'] > ito_ratio_min) & (glass_data['ratio'] < ito_ratio_max)]
glass_data = glass_data.reset_index(drop=True)
print(glass_data.shape)

# further control the dataset so that all width and length in glass can fal into the range of ITO's
ito_width_min = ito_data['width'].min()
ito_width_max = ito_data['width'].max()
ito_length_min = ito_data['length'].min()
ito_length_max = ito_data['length'].max()

glass_data = glass_data[(glass_data['width'] > ito_width_min) & (glass_data['width'] < ito_width_max)]
glass_data = glass_data[(glass_data['length'] > ito_length_min) & (glass_data['length'] < ito_length_max)]
glass_data = glass_data.reset_index(drop=True)
print(glass_data.shape)

# Randomly sample 1000 rows from each dataframe
glass_data = glass_data.sample(n=1000, random_state=42)
glass_data = glass_data.reset_index(drop=True)
ito_data = ito_data.sample(n=1000, random_state=42)
ito_data = ito_data.reset_index(drop=True)
print(glass_data.shape)

In [5]:
def calc_dif(idx, ito_data, glass_data):
    Eres_glass = glass_data['eres'][idx]
    Linewidth_glass = glass_data['linewidth'][idx]
    AR_glass = glass_data['ratio'][idx]

    upper_min = ito_data[ito_data['ratio'] > AR_glass]['ratio'].min()
    lower_max = ito_data[ito_data['ratio'] < AR_glass]['ratio'].max()

    if abs(upper_min - AR_glass) > abs(lower_max - AR_glass):
        AR_ito_closest = lower_max
    else:
        AR_ito_closest = upper_min

    Eres_ito = ito_data[ito_data['ratio'] == AR_ito_closest]['eres']
    Linewidth_ito = ito_data[ito_data['ratio'] == AR_ito_closest]['linewidth']
    dEres_IG = Eres_ito - Eres_glass
    dLinewidth_IG = Linewidth_ito - Linewidth_glass
    return dEres_IG, dLinewidth_IG


def calculate_mean(lst):
    return sum(lst) / len(lst)


def numerical_method(ito_data, glass_data, target_nsub):
    ito_nsub = 1.975
    glass_nsub = 1.52
    dnsub_IG = ito_nsub - glass_nsub
    dEres_IG_list = []
    dLinewidth_IG_list = []
    for idx in range(glass_data.shape[0]):
        dEres_IG, dLinewidth_IG = calc_dif(idx, ito_data, glass_data)
        if (dEres_IG.shape == (1,)) and (dLinewidth_IG.shape == (1,)): 
            dEres_IG_list.append(dEres_IG.values)
            dLinewidth_IG_list.append(dLinewidth_IG.values)

    dEres_IG_mean = calculate_mean(dEres_IG_list)
    dLinewidth_IG_mean = calculate_mean(dLinewidth_IG_list)
    C = dEres_IG_mean / dnsub_IG
    D = dLinewidth_IG_mean / dnsub_IG
    target_Eres = C*(target_nsub - glass_nsub) + glass_data['eres']
    target_Linewidth = D*(target_nsub - glass_nsub) + glass_data['linewidth']
    return target_Eres, target_Linewidth

In [9]:
glass_data.head()

,eres,linewidth,imax,width,length
0,2.002197,0.157153,1.690000e-15,30.51,69.67
1,1.707124,0.100981,1.110000e-14,29.08,96.84
2,1.808381,0.105667,5.520000e-15,27.39,83.27
3,1.622688,0.090948,1.250000e-14,27.05,100.63
4,1.876886,0.121050,2.210000e-15,25.83,71.47


In [7]:
# glass_data = glass_data.sample(1000, random_state=42)
# define input and output
# X_glass = glass_data.drop(['width', 'length', 'imax', 'ratio', 'volume'], axis=1)
# yw_glass = glass_data['width']
# yl_glass = glass_data['length']

X_glass = combined_data.drop(['width', 'length', 'imax', 'ratio', 'volume'], axis=1)
yw_glass = combined_data['width']
yl_glass = combined_data['length']

# ito_data = ito_data.sample(1000, random_state=42)
# define input and output
X_ito = ito_data.drop(['width', 'length', 'imax', 'ratio'], axis=1)
yw_ito = ito_data['width']
yl_ito = ito_data['length']

In [8]:
ITO_nsub = 1.96
ITO_Eres, ITO_Linewidth = numerical_method(ito_data, glass_data, target_nsub=ITO_nsub)

X_glass['eres'] = ITO_Eres
X_glass['linewidth'] = ITO_Linewidth
X_glass

KeyError: 'ratio'